In [1]:
#Combine features

In [319]:
import pandas as pd

In [320]:
df = pd.read_csv('hatespeech_processed.csv')

In [323]:
df.head()

,ID,data,label
0,319399851215433729,oh yeah colin smash girls,sexism
1,320817818222358529,insane keep bringing people back show end,none
2,324114200450437120,eagle shit show comebacks johnny farnham ok back,none
3,326286656854454273,sigh oh colin,none
4,381988216292655104,rt day15 swear sexist honestly cannot stand wo...,sexism


In [324]:
from sklearn.model_selection import train_test_split

In [325]:
train_x, valid_x, train_y, valid_y = train_test_split(df['data'],df['label'])

In [326]:
df = df.dropna()

In [327]:
#sentiment and tfidf 
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
import xgboost
var_t = train_y
var_v = valid_y
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


#ngram level tf-idf
tfidf_ngram = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(2,3) ,max_features=5000)
tfidf_ngram.fit(df['data'])
xtrain_tfidf_ngram = tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_ngram.transform(valid_x)

from keras.preprocessing import text, sequence
token = text.Tokenizer()
token.fit_on_texts(df['data'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [105]:
from sklearn.linear_model import LogisticRegression

In [106]:
clf = LogisticRegression(random_state=0, solver='lbfgs',
                       multi_class='multinomial').fit(xtrain_tfidf, train_y)

In [107]:
prediction = clf.predict(xvalid_tfidf)

In [108]:
metrics.accuracy_score(prediction, valid_y)

0.8145628621819098

In [109]:
from sklearn.metrics import f1_score
f1_score(prediction, valid_y,average=None)

array([0.87461565, 0.6878825 , 0.61878453])

In [110]:
from sklearn.metrics import classification_report
print classification_report(prediction, valid_y)

              precision    recall  f1-score   support

           0       0.94      0.82      0.87      3138
           1       0.59      0.82      0.69       341
           2       0.50      0.80      0.62       490

   micro avg       0.81      0.81      0.81      3969
   macro avg       0.68      0.81      0.73      3969
weighted avg       0.86      0.81      0.83      3969



In [48]:
# 0 - None
# 1 - Racism
# 2 - Sexism

In [49]:
#tfidf plus polarity

In [73]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
analyser = SentimentIntensityAnalyzer()

In [111]:
dataset_train = pd.DataFrame(train_x)
dataset_valid = pd.DataFrame(valid_x)


dataset_train['comp'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_train['pos'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_train['neu'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_train['neg'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

dataset_valid['comp'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_valid['pos'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_valid['neu'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_valid['neg'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

In [112]:
type(dataset_train['comp'])

pandas.core.series.Series

In [151]:
xtrain_tfidf_con = pd.DataFrame(np.concatenate((xtrain_tfidf.toarray(), dataset_train[['comp','pos','neu','neg']].values),axis=1))

In [152]:
xvalid_tfidf_con = pd.DataFrame(np.concatenate((xvalid_tfidf.toarray(), dataset_valid[['comp','pos','neu','neg']].values),axis=1))

In [153]:
xtrain_tfidf_rev = pd.DataFrame(np.concatenate(( dataset_train[['comp','pos','neu','neg']].values, xtrain_tfidf.toarray()),axis=1))

In [154]:
xvalid_tfidf_rev = pd.DataFrame(np.concatenate((dataset_valid[['comp','pos','neu','neg']].values,xvalid_tfidf.toarray()),axis=1))

In [155]:
xtrain_tfidf

<11905x5000 sparse matrix of type '<type 'numpy.float64'>'
	with 78693 stored elements in Compressed Sparse Row format>

In [156]:
dataset_train[['comp']].shape

(11905, 1)

In [157]:
clf_new = LogisticRegression(random_state=0, solver='lbfgs',
                       multi_class='multinomial').fit(xtrain_tfidf_con, train_y)

prediction_new = clf_new.predict(xvalid_tfidf_con)
print classification_report(prediction_new, valid_y)

              precision    recall  f1-score   support

           0       0.94      0.82      0.87      3126
           1       0.61      0.81      0.69       354
           2       0.50      0.80      0.61       489

   micro avg       0.81      0.81      0.81      3969
   macro avg       0.68      0.81      0.73      3969
weighted avg       0.86      0.81      0.83      3969



In [182]:
from sklearn import svm
svm_ = svm.SVC().fit(xtrain_tfidf_con, train_y)
prediction_new = clf_new.predict(xvalid_tfidf_con)
print classification_report(prediction_new, valid_y)

/Users/i504969/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.94      0.82      0.87      3126
           1       0.61      0.81      0.69       354
           2       0.50      0.80      0.61       489

   micro avg       0.81      0.81      0.81      3969
   macro avg       0.68      0.81      0.73      3969
weighted avg       0.86      0.81      0.83      3969



In [183]:
def train_model(classifier, feature_vector_train,label, feature_vector_valid, is_neural_net = False):
    classifier.fit(feature_vector_train, label)
    prediction = classifier.predict(feature_vector_valid)
    from sklearn.metrics import confusion_matrix
    if not is_neural_net:
        print(confusion_matrix(valid_y, prediction))
    if is_neural_net:
        prediction = prediction.argmax(axis=-1)
    print(classification_report(prediction, valid_y))
    return metrics.accuracy_score(prediction, valid_y), f1_score(prediction, valid_y,average=None)


In [187]:
clf = linear_model.LogisticRegression(verbose=1)
#accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
accuracy, f1 = train_model(clf, xtrain_tfidf_con, train_y, xvalid_tfidf_con)
print("Logistic Regression (TDIDF)", accuracy, f1)

[LibLinear][[2577   64   75]
 [ 195  279    2]
 [ 426    0  351]]
              precision    recall  f1-score   support

           0       0.95      0.81      0.87      3198
           1       0.59      0.81      0.68       343
           2       0.45      0.82      0.58       428

   micro avg       0.81      0.81      0.81      3969
   macro avg       0.66      0.81      0.71      3969
weighted avg       0.86      0.81      0.82      3969

('Logistic Regression (TDIDF)', 0.8080120937263794, array([0.87149138, 0.68131868, 0.58257261]))


In [189]:
svm_ = svm.SVC(C=1e5)
accuracy, f1 = train_model(svm_, xtrain_tfidf_con, train_y, xvalid_tfidf_con)

[[2290  133  293]
 [ 161  307    8]
 [ 320    3  454]]
              precision    recall  f1-score   support

           0       0.84      0.83      0.83      2771
           1       0.64      0.69      0.67       443
           2       0.58      0.60      0.59       755

   micro avg       0.77      0.77      0.77      3969
   macro avg       0.69      0.71      0.70      3969
weighted avg       0.77      0.77      0.77      3969



In [158]:
unique, counts = np.unique(valid_y, return_counts=True)

In [159]:
unique, counts = np.unique(valid_y, return_counts=True)
dict(zip(unique, counts))

{0: 2716, 1: 476, 2: 777}

In [321]:
#Trying word2vec

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [322]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['data']), tags=[r.label]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['data']), tags=[r.label]), axis=1)

TypeError: ('expected string or buffer', u'occurred at index 8128')

In [174]:
train_tagged[2]

TaggedDocument(words=['eagle', 'shit', 'show', 'comebacks', 'johnny', 'farnham', 'ok', 'back'], tags=['none'])

In [175]:
import multiprocessing
cores = multiprocessing.cpu_count()


In [167]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 11111/11111 [00:00<00:00, 609754.30it/s]


defaultdict(int, {})

In [168]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 11111/11111 [00:00<00:00, 637828.12it/s]


CPU times: user 15.2 s, sys: 6.54 s, total: 21.7 s
Wall time: 14.1 s


In [170]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [181]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.6764644131849674
Testing F1 score: 0.58057488118


In [190]:
#Do PCA with data and check for results

In [329]:
from sklearn.decomposition import PCA, TruncatedSVD


pca = TruncatedSVD(n_components=3500)
principalComponents = pca.fit_transform(xtrain_tfidf)
principalDf = pd.DataFrame(data = principalComponents)


In [330]:
#Do PCA on combine dataset as well?

pca_con = TruncatedSVD(n_components=2000)
principalComponents_con = pca.fit_transform(xtrain_tfidf_con)
principalDf_con = pd.DataFrame(data = principalComponents)

KeyboardInterrupt: 

In [ ]:
#Check impact of doing pca on results
pca_valid = TruncatedSVD(n_components=3500)

principalComponents_valid = pca_valid.fit_transform(xvalid_tfidf)

clf = linear_model.LogisticRegression()
clf.fit(principalComponents, train_y)
p = clf.predict(principalComponents_valid)

In [ ]:
print classification_report(p,valid_y)

In [202]:
#pca is not working because it's considering only features of neutral to be important while prediction
# let's look into balancing dataset and then again performing same operation

In [331]:
df_minority_racism = df[df['label']=='racism']

In [332]:
df_minority_sexism = df[df['label']=='sexism']

In [333]:
df_majority_neu =  df[df['label']=='none']

In [334]:
df_minority_sexism.shape

(3158, 3)

In [335]:
df_minority_racism.shape

(1935, 3)

In [336]:
df_minority_racism = df[df['label']=='racism']
df_minority_sexism = df[df['label']=='sexism']
df_majority_neu =  df[df['label']=='none']
from sklearn.utils import resample

df_minority_upsampled = resample(df_minority_sexism, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=500) # reproducible results
df_minority_upsampled_racism = resample(df_minority_racism, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=500) # reproducible results

df_new = pd.concat([df_majority_neu, df_minority_upsampled_racism, df_minority_upsampled])


In [337]:
df_minority_upsampled.shape

(5000, 3)

In [338]:
df_minority_upsampled_racism = resample(df_minority_racism, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=500) # reproducible results



In [339]:
df_majority_neu.shape

(10781, 3)

In [340]:
df_new = pd.concat([df_majority_neu, df_minority_upsampled_racism, df_minority_upsampled])

In [341]:
df_new.shape

(20781, 3)

In [342]:
df.shape

(15874, 3)

In [343]:
df_new = df_new.dropna()
train_x, valid_x, train_y, valid_y = train_test_split(df_new['data'],df_new['label'])


var_t = train_y
var_v = valid_y
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


#ngram level tf-idf
tfidf_ngram = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(2,3) ,max_features=5000)
tfidf_ngram.fit(df['data'])
xtrain_tfidf_ngram = tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_ngram.transform(valid_x)


In [353]:
pickle.dump(tfidf_vect, open("tfidf.pk", "wb"))


In [354]:
tfidf_vect_new = open("tfidf.pickle","wb")

In [356]:
tfidf_vect_new.transform(train_x)

AttributeError: 'file' object has no attribute 'transform'

In [269]:
clf = linear_model.LogisticRegression(C=1)
#accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("Logistic Regression (TDIDF)", accuracy, f1)

[[2497  108  113]
 [ 186 1078   11]
 [ 411    0  792]]
              precision    recall  f1-score   support

           0       0.92      0.81      0.86      3094
           1       0.85      0.91      0.88      1186
           2       0.66      0.86      0.75       916

   micro avg       0.84      0.84      0.84      5196
   macro avg       0.81      0.86      0.83      5196
weighted avg       0.86      0.84      0.84      5196

('Logistic Regression (TDIDF)', 0.840454195535027, array([0.85925671, 0.87606664, 0.74752242]))


In [344]:
from sklearn.model_selection import GridSearchCV

parameters = {'penalty':['l1','l2'],'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
clf_log = linear_model.LogisticRegression()
clf = GridSearchCV(clf_log, parameters, cv=5)

# accuracy, f1 = train_model(clf_log, train_x_count, train_y, valid_x_count)
# print("Logistic Regression (Count Vectors)",accuracy, f1)
accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("Logistic Regression)", accuracy, f1)
accuracy, f1 = train_model(clf, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("Logistic Regression (TDIDF-ngram)", accuracy, f1)

KeyboardInterrupt: 

In [277]:
clf.best_params_

{'C': 10, 'penalty': 'l2'}

In [348]:
clf = linear_model.LogisticRegression(C=10)
clf.fit(xtrain_tfidf, train_y)
prediction = clf.predict(xvalid_tfidf)
accuracy, f1 = metrics.accuracy_score(prediction, valid_y), f1_score(prediction, valid_y,average=None)

#accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
#accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("Logistic Regression (TDIDF)", accuracy, f1)

('Logistic Regression (TDIDF)', 0.8733641262509623, array([0.87725904, 0.92210212, 0.81017939]))


In [358]:
import pickle
filename = 'lrmodel.sav'
pickle.dump(clf, open(filename, 'wb'))
 


In [357]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [255]:
#NEW DATA
clf = linear_model.LogisticRegression()
#accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("Logistic Regression (TDIDF)", accuracy, f1)

[[2546   74   87]
 [ 154  310    3]
 [ 452    0  570]]
              precision    recall  f1-score   support

           0       0.94      0.81      0.87      3152
           1       0.66      0.81      0.73       384
           2       0.56      0.86      0.68       660

   micro avg       0.82      0.82      0.82      4196
   macro avg       0.72      0.83      0.76      4196
weighted avg       0.85      0.82      0.83      4196

('Logistic Regression (TDIDF)', 0.8164918970448046, array([0.86909029, 0.72855464, 0.67776457]))


In [244]:
clf = linear_model.LogisticRegression()
#accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
accuracy, f1 = train_model(clf, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("Logistic Regression Ngram (TDIDF)", accuracy, f1)

[[2553   85   55]
 [ 496  750    7]
 [ 695    3  552]]
              precision    recall  f1-score   support

           0       0.95      0.68      0.79      3744
           1       0.60      0.89      0.72       838
           2       0.44      0.90      0.59       614

   micro avg       0.74      0.74      0.74      5196
   macro avg       0.66      0.83      0.70      5196
weighted avg       0.83      0.74      0.76      5196

('Logistic Regression Ngram (TDIDF)', 0.7419168591224018, array([0.79322666, 0.71736011, 0.59227468]))


In [245]:
svm_ = svm.SVC(C=0.1,kernel='linear')
accuracy, f1 = train_model(svm_, xtrain_tfidf, train_y, xvalid_tfidf)

[[2546   95   52]
 [ 319  934    0]
 [ 708    0  542]]
              precision    recall  f1-score   support

           0       0.95      0.71      0.81      3573
           1       0.75      0.91      0.82      1029
           2       0.43      0.91      0.59       594

   micro avg       0.77      0.77      0.77      5196
   macro avg       0.71      0.84      0.74      5196
weighted avg       0.85      0.77      0.79      5196



In [256]:
#NEW DATA
svm_ = svm.SVC(C=0.1,kernel='linear')
accuracy, f1 = train_model(svm_, xtrain_tfidf, train_y, xvalid_tfidf)

[[2618   62   27]
 [ 214  253    0]
 [ 680    0  342]]
              precision    recall  f1-score   support

           0       0.97      0.75      0.84      3512
           1       0.54      0.80      0.65       315
           2       0.33      0.93      0.49       369

   micro avg       0.77      0.77      0.77      4196
   macro avg       0.61      0.83      0.66      4196
weighted avg       0.88      0.77      0.80      4196



In [246]:
print("SVM (TDIDF)", accuracy, f1)

('SVM (TDIDF)', 0.7740569668976135, array([0.81263964, 0.81858019, 0.58785249]))


In [257]:
print("SVM (TDIDF)", accuracy, f1)

('SVM (TDIDF)', 0.7657292659675882, array([0.841936  , 0.64705882, 0.49173257]))


In [240]:
print("SVM (TDIDF)", accuracy, f1)

('SVM (TDIDF)', 0.7732426303854876, array([0.85393989, 0.55238095, 0.45419847]))


In [231]:
print("SVM (TDIDF)", accuracy, f1)

('SVM (TDIDF)', 0.7588813303099018, array([0.82714469, 0.65116279, 0.58098361]))


In [232]:
pca = TruncatedSVD(n_components=3500)
principalComponents = pca.fit_transform(xtrain_tfidf)
principalDf = pd.DataFrame(data = principalComponents)



pca_valid = TruncatedSVD(n_components=3500)

principalComponents_valid = pca_valid.fit_transform(xvalid_tfidf)

clf = linear_model.LogisticRegression()
clf.fit(principalComponents, train_y)
p = clf.predict(principalComponents_valid)

In [234]:
print classification_report(p, valid_y)

              precision    recall  f1-score   support

           0       0.88      0.73      0.80      3248
           1       0.23      0.51      0.32       225
           2       0.32      0.51      0.40       496

   micro avg       0.69      0.69      0.69      3969
   macro avg       0.48      0.58      0.51      3969
weighted avg       0.78      0.69      0.72      3969



In [237]:
svm_ = svm.SVC(C=1)
accuracy, f1 = train_model(svm_, principalDf, train_y, principalComponents_valid)
print ("PCA SVM Classifier Accuracy and f1 score: ",accuracy, f1)

[[2688    0    0]
 [ 491    0    0]
 [ 790    0    0]]
              precision    recall  f1-score   support

           0       1.00      0.68      0.81      3969
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

   micro avg       0.68      0.68      0.68      3969
   macro avg       0.33      0.23      0.27      3969
weighted avg       1.00      0.68      0.81      3969

('PCA SVM Classifier Accuracy and f1 score: ', 0.6772486772486772, array([0.80757098, 0.        , 0.        ]))


In [279]:
#Training and testing Logistic Regression model

In [280]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(df['data'],df['label'],test_size=0.3)

In [289]:
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


In [293]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(C=10)
logmodel.fit(xtrain_tfidf,train_y)
predictions = logmodel.predict(xvalid_tfidf)

In [294]:
print(classification_report(valid_y,predictions))


              precision    recall  f1-score   support

           0       0.88      0.89      0.88      2718
           1       0.91      0.92      0.92      1275
           2       0.83      0.80      0.82      1203

   micro avg       0.88      0.88      0.88      5196
   macro avg       0.87      0.87      0.87      5196
weighted avg       0.88      0.88      0.88      5196



In [298]:
print metrics.accuracy_score(predictions, valid_y)
print f1_score(predictions, valid_y,average=None)



0.8756735950731331
[0.8823745  0.91630985 0.81606765]


In [295]:
unique, counts = np.unique(valid_y, return_counts=True)
dict(zip(unique, counts))

{0: 2718, 1: 1275, 2: 1203}